Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [0]:
# convert SALE_DATE to DateTime format

df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], format='%m/%d/%Y')

In [0]:
# convert LAND_SQUARE_FEET to numeric format

df['LAND_SQUARE_FEET'] = df['LAND_SQUARE_FEET'].str.replace(',', '')
df['LAND_SQUARE_FEET'] = pd.to_numeric(df['LAND_SQUARE_FEET'], errors='ignore')

In [203]:
df.isnull().sum()

BOROUGH                               0
NEIGHBORHOOD                          0
BUILDING_CLASS_CATEGORY               0
TAX_CLASS_AT_PRESENT                  1
BLOCK                                 0
LOT                                   0
EASE-MENT                         23040
BUILDING_CLASS_AT_PRESENT             1
ADDRESS                               0
APARTMENT_NUMBER                  17839
ZIP_CODE                              1
RESIDENTIAL_UNITS                     1
COMMERCIAL_UNITS                      1
TOTAL_UNITS                           1
LAND_SQUARE_FEET                     53
GROSS_SQUARE_FEET                     1
YEAR_BUILT                           35
TAX_CLASS_AT_TIME_OF_SALE             0
BUILDING_CLASS_AT_TIME_OF_SALE        0
SALE_PRICE                            0
SALE_DATE                             0
dtype: int64

In [0]:
# Use a subset of the data where BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS' and the sale price was more than 100 thousand and less than 2 million.

df_subset = df[(df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS') & (df['SALE_PRICE'].between(100000, 2000000))]

In [0]:
# Do one-hot encoding of categorical features.

import category_encoders as ce

category_features = ['NEIGHBORHOOD',
                     'BUILDING_CLASS_AT_TIME_OF_SALE', 
                     'BUILDING_CLASS_AT_PRESENT',
                     'TAX_CLASS_AT_PRESENT',
                     'BOROUGH']

encoder = ce.one_hot.OneHotEncoder(use_cat_names=True)
cols_encoded = encoder.fit_transform(df_subset[category_features])

df_enc = df_subset.drop(category_features+['BUILDING_CLASS_CATEGORY','ADDRESS', 'APARTMENT_NUMBER'], axis=1)
df_enc[cols_encoded.columns] = cols_encoded

In [0]:
# Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.

mask = (df_enc['SALE_DATE'].dt.year == 2019) & (df_enc['SALE_DATE'].dt.month.between(1, 3))
df_train = df_enc[mask]

mask2 = (df_enc['SALE_DATE'].dt.year == 2019) & (df_enc['SALE_DATE'].dt.month == 4)
df_test = df_enc[mask2]

In [0]:
features = ['BLOCK', 'LOT', 'EASE-MENT', 'ZIP_CODE', 'RESIDENTIAL_UNITS',
       'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'LAND_SQUARE_FEET',
       'GROSS_SQUARE_FEET', 'YEAR_BUILT', 'TAX_CLASS_AT_TIME_OF_SALE',
       'SALE_PRICE', 'NEIGHBORHOOD_OTHER',
       'NEIGHBORHOOD_FLUSHING-NORTH', 'NEIGHBORHOOD_EAST NEW YORK',
       'NEIGHBORHOOD_BEDFORD STUYVESANT', 'NEIGHBORHOOD_FOREST HILLS',
       'NEIGHBORHOOD_BOROUGH PARK', 'NEIGHBORHOOD_ASTORIA',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A9',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A1',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A5',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A0',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A2',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A3',
       'BUILDING_CLASS_AT_TIME_OF_SALE_S1',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A4',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A6',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A8',
       'BUILDING_CLASS_AT_TIME_OF_SALE_S0', 'BUILDING_CLASS_AT_PRESENT_A9',
       'BUILDING_CLASS_AT_PRESENT_A1', 'BUILDING_CLASS_AT_PRESENT_A5',
       'BUILDING_CLASS_AT_PRESENT_A0', 'BUILDING_CLASS_AT_PRESENT_A2',
       'BUILDING_CLASS_AT_PRESENT_A3', 'BUILDING_CLASS_AT_PRESENT_S1',
       'BUILDING_CLASS_AT_PRESENT_A4', 'BUILDING_CLASS_AT_PRESENT_A6',
       'BUILDING_CLASS_AT_PRESENT_A8', 'BUILDING_CLASS_AT_PRESENT_B2',
       'BUILDING_CLASS_AT_PRESENT_S0', 'BUILDING_CLASS_AT_PRESENT_B3',
       'TAX_CLASS_AT_PRESENT_1', 'TAX_CLASS_AT_PRESENT_1D', 'BOROUGH_3',
       'BOROUGH_4', 'BOROUGH_2', 'BOROUGH_5', 'BOROUGH_1']

In [0]:
X_train = df_train[features]
Y_train = df_train['SALE_PRICE']

X_test = df_test[features]
Y_test = df_test['SALE_PRICE']

In [195]:
X_train.dtypes

BLOCK                                  int64
LOT                                    int64
EASE-MENT                            float64
ZIP_CODE                             float64
RESIDENTIAL_UNITS                    float64
COMMERCIAL_UNITS                     float64
TOTAL_UNITS                          float64
LAND_SQUARE_FEET                      object
GROSS_SQUARE_FEET                    float64
YEAR_BUILT                           float64
TAX_CLASS_AT_TIME_OF_SALE              int64
SALE_PRICE                             int64
NEIGHBORHOOD_OTHER                     int64
NEIGHBORHOOD_FLUSHING-NORTH            int64
NEIGHBORHOOD_EAST NEW YORK             int64
NEIGHBORHOOD_BEDFORD STUYVESANT        int64
NEIGHBORHOOD_FOREST HILLS              int64
NEIGHBORHOOD_BOROUGH PARK              int64
NEIGHBORHOOD_ASTORIA                   int64
BUILDING_CLASS_AT_TIME_OF_SALE_A9      int64
BUILDING_CLASS_AT_TIME_OF_SALE_A1      int64
BUILDING_CLASS_AT_TIME_OF_SALE_A5      int64
BUILDING_C

In [0]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [194]:
# Do feature selection with SelectKBest.

for k in range(1, df_train.shape[1]+1):
  selector = SelectKBest(score_func=f_regression, k=k)
  X_train_kbest = selector.fit_transform(X_train, Y_train)
  X_test_Kbest = selector.transform(X_test)
  model = LinearRegression()
  model.fit(X_train_kbest, Y_train)
  Y_pred = model.predict(X_test_Kbest)
  mse = metrics.mean_squared_error(Y_test, Y_pred)
  print(k, features)
  print(f'MSE: ${mse:0f}')

ValueError: ignored